In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [84]:
data = pd.read_csv("data/weatherAUS.csv")

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [86]:
data.isnull().sum().sum()

343248

In [87]:
data.isnull().sum() / data.shape[0]

Date             0.000000
Location         0.000000
MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
RainTomorrow     0.022460
dtype: float64

In [88]:
data = data.drop(columns=['Evaporation', 'Sunshine', 'Cloud3pm'])

In [89]:
data['RainToday'] = data['RainToday'].apply(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else x)
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else x)

In [90]:
round(data['RainToday'].mean(), 2)

0.22

In [91]:
data['Month'] = pd.to_datetime(data['Date']).dt.month
data = data.drop(columns='Date')

In [92]:
data[data['RainToday'] == 1].groupby(data['Month'])['Month'].count() / data.groupby(data['Month'])['Month'].count() 

Month
1     0.184875
2     0.201612
3     0.211885
4     0.212208
5     0.217255
6     0.257569
7     0.265198
8     0.247589
9     0.223368
10    0.192966
11    0.206959
12    0.207699
Name: Month, dtype: float64

In [93]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data = pd.get_dummies(data, columns=categoricals)

In [94]:
data.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0


In [95]:
data = data.dropna(axis=0)
X = data.drop(columns='RainTomorrow')
y = data['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31, test_size=0.3)

In [96]:
round(y_test.mean(), 2)

0.23

In [97]:
from sklearn import linear_model #линейные модели
from sklearn.metrics import roc_auc_score


In [98]:
#Создаём объект класса LogisticRegression
log_reg_2d = linear_model.LogisticRegression(random_state=42)
#Обучаем модель, минизируя logloss
log_reg_2d.fit(X_train, y_train)
y_pred = log_reg_2d.predict(X_test)

roc_auc_score(y_test, y_pred)

c:\Users\ak88r\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7274246249538552

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree

params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

grid_search = GridSearchCV(
    estimator= tree.DecisionTreeClassifier(
        random_state=42 #генератор случайных чисел
    ), 
    param_grid=params, 
    cv=3 
    )  
%time grid_search.fit(X_train, y_train) 
#print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test_scaled, y_test)))
y_test_pred = grid_search.predict(X_test)
print('roc_auc_score на тестовом наборе: {:.2f}'.format(roc_auc_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 52.3 s
Wall time: 52.5 s
roc_auc_score на тестовом наборе: 0.70
Наилучшие значения гиперпараметров: {'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}
